In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pyodbc
import sqlite3

In [ ]:
# Connect to SQL Server
DB = {'servername': 'LAPTOP-HSI9FLRD\\SQLEXPRESS',
      'database': 'DataWarehouseGreatOutdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] +
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()

In [ ]:
orders = pd.read_sql_query('SELECT * FROM order_details', export_conn)
orders

In [ ]:
orders = orders.loc[ :, ['Order_details_sk', 'Order_number', 'Order_method_code', 'Quantity', 'Order_detail_unit_cost', 'Order_detail_unit_price', 'Order_detail_product_number', 'Retailer_site_code']]
orders

In [ ]:
x = orders.drop('Quantity', axis=1)
y = orders.loc[:, ['Quantity']]
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size = 0.15, ## 15% of the data will be used for testing
    random_state = 48
)


In [ ]:
reg_model = linear_model.LinearRegression()
reg_model = LinearRegression().fit(x_train, y_train)
reg_model

In [ ]:
y_pred = reg_model.predict(x_test)
y_pred

In [ ]:
prediction_orders = pd.DataFrame(y_pred)
prediction_orders = prediction_orders.rename(columns={0: 'Predicted_Quantity'})
prediction_orders

In [ ]:
y_test_prediction_merge = pd.concat([y_test.reset_index()['Quantity'], prediction_orders], axis=1)
y_test_prediction_merge.loc[y_test_prediction_merge['Quantity'].notna(), :]

In [ ]:
plt.scatter(y_test_prediction_merge['Quantity'], y_test_prediction_merge['Predicted_Quantity'])
plt.xlabel('Quantity')
plt.ylabel('Predicted Quantity')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.show()

In [ ]:
mean_squared_error(y_test_prediction_merge['Quantity'], y_test_prediction_merge['Predicted_Quantity'])

In [ ]:
mean_absolute_error(y_test_prediction_merge['Quantity'], y_test_prediction_merge['Predicted_Quantity'])